<a href="https://colab.research.google.com/github/mariomartino/SDAFile/blob/main/ScriptClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


NAIVE BAYES CLASSIFICATION
---



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore");

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Machine Learning')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

myds_pandas=pd.read_csv('ClassificationData_SDA_IZ_group2.csv')

In [ ]:
Z=myds_pandas['Z_OppositeTeamDefence']
Z

Y=myds_pandas[["Y_Dehydration", "Y_Hyperthermia","Y_AvgSpeed", "Y_AvgTravelledDistance" , "Y_PressingCapability", "Y_PhysicalEndurance", "Y_MentalConcentration", "Y_EmotionalMotivation"]]
Y

,Y_Dehydration,Y_Hyperthermia,Y_AvgSpeed,Y_AvgTravelledDistance,Y_PressingCapability,Y_PhysicalEndurance,Y_MentalConcentration,Y_EmotionalMotivation
0,6.677760,8.343137,3.076781,-4.211415,12.082853,4.085970,12.530036,11.984524
1,5.366346,5.356699,2.853831,-3.011611,10.866838,-0.310016,12.807582,13.039656
2,5.117354,5.639778,-0.779392,-5.783385,5.295872,-3.343576,6.453986,6.861213
3,6.552929,10.164186,2.434366,-3.559911,12.619849,2.701936,14.445014,11.622576
4,6.518145,8.859181,1.764184,-4.980409,10.011302,0.357171,10.459209,9.989347
...,...,...,...,...,...,...,...,...
195,3.968906,2.767570,5.222294,-1.895960,10.684503,2.552538,12.762757,11.276801
196,5.136077,6.247014,-1.898779,-5.603919,5.227272,-4.161020,7.042536,7.737527
197,2.121075,2.343614,2.460799,-2.016852,9.696119,2.423240,12.209254,12.284317
198,4.089910,5.242092,-0.375904,-6.291635,5.394229,-2.707619,5.890800,7.439935


In [ ]:

YY_train, YY_test, ZZ_train, ZZ_test= train_test_split(Y,Z, test_size=0.8)


In [ ]:
###Classification
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
ZZ_estimated=gnb.fit(YY_train, ZZ_train).predict(YY_test)

#Evaluate Results
print('Confusion Matrix:')
print(confusion_matrix(ZZ_test, ZZ_estimated))
print('\nClassifier Performance Metrics')
print(classification_report(ZZ_test, ZZ_estimated))

Confusion Matrix:
[[ 42   3]
 [  6 109]]

Classifier Performance Metrics
              precision    recall  f1-score   support

           0       0.88      0.93      0.90        45
           1       0.97      0.95      0.96       115

    accuracy                           0.94       160
   macro avg       0.92      0.94      0.93       160
weighted avg       0.95      0.94      0.94       160



In [ ]:
 from sklearn.linear_model import LogisticRegression, SGDClassifier
 lr = SGDClassifier('log')
 #Model fitting
 lr.fit(YY_train, ZZ_train)
 #Predict  class labels for sample in X
 ZZ_estimated = lr.predict(YY_test)


  #Evaluate Results
print('Confusion Matrix:')
print(confusion_matrix(ZZ_test, ZZ_estimated))
print('\nClassifier Performance Metrics')
print(classification_report(ZZ_test, ZZ_estimated))

Confusion Matrix:
[[ 43   2]
 [  6 109]]

Classifier Performance Metrics
              precision    recall  f1-score   support

           0       0.88      0.96      0.91        45
           1       0.98      0.95      0.96       115

    accuracy                           0.95       160
   macro avg       0.93      0.95      0.94       160
weighted avg       0.95      0.95      0.95       160



In [ ]:
ZZ_test = np.array(ZZ_test)
err=np.argwhere(ZZ_test!=ZZ_estimated).reshape((-1,))
YY_test_np=np.array(YY_test)
for i in range (err.size):
  print(YY_test_np[i])


[ 5.9707921   8.48634254  0.88455009 -4.07865504 10.02719706  1.96854964
  9.66738503 10.65931404]
[ 5.00347662  5.75843569  6.60855897 -3.73618901 12.69786996  3.69908889
 11.76841486 11.04176883]
[ 6.99998985  7.31436562  0.17017478 -5.27618962  7.62860164 -1.67940773
  9.58055171  9.19821979]
[ 4.88782803  7.69574783 -1.78165614 -6.26873083  7.10705299 -3.64995494
  9.38128146 10.03507479]
[ 8.06549289 10.07065448  3.36968742 -4.81187457 11.21669975  2.2422762
 12.27084269  9.19749232]
[ 6.06146506  7.63810458  4.44257886 -4.65029945 12.05237065  3.32335869
 12.36245481 10.64132859]
[ 7.0758703  11.49018887 -3.80257847 -7.24131584  7.17874768 -3.64229148
 10.43778744 10.27967401]
[ 5.23282886  7.78947819  1.03287296 -3.38535945  9.53087716  3.02195789
  8.54349935  9.61279273]
[ 5.72619798  8.67866086 -3.65420425 -5.95046455  4.93371956 -3.73783784
  7.8010601   7.44056997]


In [ ]:
#PCA PROVA
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA


# fits PCA, transforms data and fits the decision tree classifier
# on the transformed data
for i in range (8):
  pipe = Pipeline([('pca', PCA(n_components= i+1)),('Logistic Regression', SGDClassifier('log'))])
  pipe.fit(YY_train, ZZ_train)
  ZZ_estimated= pipe.predict(YY_test)
  print('current_components: ' , i+1 )
  #Evaluate Results
  print('Confusion Matrix:')
  print(confusion_matrix(ZZ_test, ZZ_estimated))
  print('\nClassifier Performance Metrics')
  print(classification_report(ZZ_test, ZZ_estimated))



current_components:  1
Confusion Matrix:
[[ 37   8]
 [  4 111]]

Classifier Performance Metrics
              precision    recall  f1-score   support

           0       0.90      0.82      0.86        45
           1       0.93      0.97      0.95       115

    accuracy                           0.93       160
   macro avg       0.92      0.89      0.90       160
weighted avg       0.92      0.93      0.92       160

current_components:  2
Confusion Matrix:
[[ 43   2]
 [ 10 105]]

Classifier Performance Metrics
              precision    recall  f1-score   support

           0       0.81      0.96      0.88        45
           1       0.98      0.91      0.95       115

    accuracy                           0.93       160
   macro avg       0.90      0.93      0.91       160
weighted avg       0.93      0.93      0.93       160

current_components:  3
Confusion Matrix:
[[41  4]
 [21 94]]

Classifier Performance Metrics
              precision    recall  f1-score   support

       